# Clase Práctica 6: Planificación
---

La clase `PlanningProblem` se va a utilizar para representar problemas de planificación. Los siguientes atributos son esenciales para poder definir un problema:
* un estado inicial
* un conjunto de objetivos
* un conjunto de acciones viables que se pueden ejecutar en el espacio de búsqueda del problema

`initial` es una expresión que forma la base de conocimiento inicial para el problema.
<br>
`goals` es una expresión que indica las metas a alcanzar por el problema.
<br>
`actions` contiene una lista de objetos `Action` que pueden ejecutarse en el espacio de búsqueda del problema.
<br>
`goal_test` comprueba si se ha alcanzado el objetivo.
<br>
`act` representa la acción dada y actualiza el estado actual.
<br>

In [ ]:
import itertools
import numpy as np
import search
from utils import *
from logic import *
from search import *
from planning import *

In [ ]:
class PlanningProblem:
    """
    Planning Domain Definition Language (PlanningProblem) used to define a search problem.
    It stores states in a knowledge base consisting of first order logic statements.
    The conjunction of these logical statements completely defines a state.
    """

    def __init__(self, initial, goals, actions, domain=None):
        self.initial = self.convert(initial) if domain is None else self.convert(initial) + self.convert(domain)
        self.goals = self.convert(goals)
        self.actions = actions
        self.domain = domain

    def convert(self, clauses):
        """Converts strings into exprs"""
        if not isinstance(clauses, Expr):
            if len(clauses) > 0:
                clauses = expr(clauses)
            else:
                clauses = []
        try:
            clauses = conjuncts(clauses)
        except AttributeError:
            pass

        new_clauses = []
        for clause in clauses:
            if clause.op == '~':
                new_clauses.append(expr('Not' + str(clause.args[0])))
            else:
                new_clauses.append(clause)
        return new_clauses

    def expand_fluents(self, name=None):

        kb = None
        if self.domain:
            kb = FolKB(self.convert(self.domain))
            for action in self.actions:
                if action.precond:
                    for fests in set(action.precond).union(action.effect).difference(self.convert(action.domain)):
                        if fests.op[:3] != 'Not':
                            kb.tell(expr(str(action.domain) + ' ==> ' + str(fests)))

        objects = set(arg for clause in set(self.initial + self.goals) for arg in clause.args)
        fluent_list = []
        if name is not None:
            for fluent in self.initial + self.goals:
                if str(fluent) == name:
                    fluent_list.append(fluent)
                    break
        else:
            fluent_list = list(map(lambda fluent: Expr(fluent[0], *fluent[1]),
                                   {fluent.op: fluent.args for fluent in self.initial + self.goals +
                                    [clause for action in self.actions for clause in action.effect if
                                     clause.op[:3] != 'Not']}.items()))

        expansions = []
        for fluent in fluent_list:
            for permutation in itertools.permutations(objects, len(fluent.args)):
                new_fluent = Expr(fluent.op, *permutation)
                if (self.domain and kb.ask(new_fluent) is not False) or not self.domain:
                    expansions.append(new_fluent)

        return expansions

    def expand_actions(self, name=None):
        """Generate all possible actions with variable bindings for precondition selection heuristic"""

        has_domains = all(action.domain for action in self.actions if action.precond)
        kb = None
        if has_domains:
            kb = FolKB(self.initial)
            for action in self.actions:
                if action.precond:
                    kb.tell(expr(str(action.domain) + ' ==> ' + str(action)))

        objects = set(arg for clause in self.initial for arg in clause.args)
        expansions = []
        action_list = []
        if name is not None:
            for action in self.actions:
                if str(action.name) == name:
                    action_list.append(action)
                    break
        else:
            action_list = self.actions

        for action in action_list:
            for permutation in itertools.permutations(objects, len(action.args)):
                bindings = unify_mm(Expr(action.name, *action.args), Expr(action.name, *permutation))
                if bindings is not None:
                    new_args = []
                    for arg in action.args:
                        if arg in bindings:
                            new_args.append(bindings[arg])
                        else:
                            new_args.append(arg)
                    new_expr = Expr(str(action.name), *new_args)
                    if (has_domains and kb.ask(new_expr) is not False) or (
                            has_domains and not action.precond) or not has_domains:
                        new_preconds = []
                        for precond in action.precond:
                            new_precond_args = []
                            for arg in precond.args:
                                if arg in bindings:
                                    new_precond_args.append(bindings[arg])
                                else:
                                    new_precond_args.append(arg)
                            new_precond = Expr(str(precond.op), *new_precond_args)
                            new_preconds.append(new_precond)
                        new_effects = []
                        for effect in action.effect:
                            new_effect_args = []
                            for arg in effect.args:
                                if arg in bindings:
                                    new_effect_args.append(bindings[arg])
                                else:
                                    new_effect_args.append(arg)
                            new_effect = Expr(str(effect.op), *new_effect_args)
                            new_effects.append(new_effect)
                        expansions.append(Action(new_expr, new_preconds, new_effects))

        return expansions

    def is_strips(self):
        """
        Returns True if the problem does not contain negative literals in preconditions and goals
        """
        return (all(clause.op[:3] != 'Not' for clause in self.goals) and
                all(clause.op[:3] != 'Not' for action in self.actions for clause in action.precond))

    def goal_test(self):
        """Checks if the goals have been reached"""
        return all(goal in self.initial for goal in self.goals)

    def act(self, action):
        """
        Performs the action given as argument.
        Note that action is an Expr like expr('Remove(Glass, Table)') or expr('Eat(Sandwich)')
        """
        action_name = action.op
        args = action.args
        list_action = first(a for a in self.actions if a.name == action_name)
        if list_action is None:
            raise Exception("Action '{}' not found".format(action_name))
        if not list_action.check_precond(self.initial, args):
            raise Exception("Action '{}' pre-conditions not satisfied".format(action))
        self.initial = list_action(self.initial, args).clauses


## Acción
Para poder modelar correctamente un problema de planificación, es fundamental poder representar una Acción. Cada acción que modelamos requiere al menos tres cosas:
* condiciones previas que debe cumplir la acción
* los efectos de ejecutar la acción
* alguna expresión que representa la acción

Esta clase representa una acción dada la expresión, las condiciones previas y sus efectos.
Una lista `precond` almacena las condiciones previas de la acción y una lista `effect` almacena sus efectos.
El método `convert` toma una cadena de entrada, la analiza, elimina las conjunciones si las hay y devuelve una lista de objetos `Expr`.
El método `check_precond` verifica si las condiciones previas para esa acción son válidas.
El método `act` lleva a cabo la acción sobre la base de conocimiento dada.

In [ ]:
class Action:
    """
    Defines an action schema using preconditions and effects.
    Use this to describe actions in PlanningProblem.
    action is an Expr where variables are given as arguments(args).
    Precondition and effect are both lists with positive and negative literals.
    Negative preconditions and effects are defined by adding a 'Not' before the name of the clause
    Example:
    precond = [expr("Human(person)"), expr("Hungry(Person)"), expr("NotEaten(food)")]
    effect = [expr("Eaten(food)"), expr("Hungry(person)")]
    eat = Action(expr("Eat(person, food)"), precond, effect)
    """

    def __init__(self, action, precond, effect, domain=None):
        if isinstance(action, str):
            action = expr(action)
        self.name = action.op
        self.args = action.args
        self.precond = self.convert(precond) if domain is None else self.convert(precond) + self.convert(domain)
        self.effect = self.convert(effect)
        self.domain = domain

    def __call__(self, kb, args):
        return self.act(kb, args)

    def __repr__(self):
        return '{}'.format(Expr(self.name, *self.args))

    def convert(self, clauses):
        """Converts strings into Exprs"""
        if isinstance(clauses, Expr):
            clauses = conjuncts(clauses)
            for i in range(len(clauses)):
                if clauses[i].op == '~':
                    clauses[i] = expr('Not' + str(clauses[i].args[0]))

        elif isinstance(clauses, str):
            clauses = clauses.replace('~', 'Not')
            if len(clauses) > 0:
                clauses = expr(clauses)

            try:
                clauses = conjuncts(clauses)
            except AttributeError:
                pass

        return clauses

    def relaxed(self):
        """
        Removes delete list from the action by removing all negative literals from action's effect
        """
        return Action(Expr(self.name, *self.args), self.precond,
                      list(filter(lambda effect: effect.op[:3] != 'Not', self.effect)))

    def substitute(self, e, args):
        """Replaces variables in expression with their respective Propositional symbol"""

        new_args = list(e.args)
        for num, x in enumerate(e.args):
            for i, _ in enumerate(self.args):
                if self.args[i] == x:
                    new_args[num] = args[i]
        return Expr(e.op, *new_args)

    def check_precond(self, kb, args):
        """Checks if the precondition is satisfied in the current state"""

        if isinstance(kb, list):
            kb = FolKB(kb)
        for clause in self.precond:
            if self.substitute(clause, args) not in kb.clauses:
                return False
        return True

    def act(self, kb, args):
        """Executes the action on the state's knowledge base"""

        if isinstance(kb, list):
            kb = FolKB(kb)

        if not self.check_precond(kb, args):
            raise Exception('Action pre-conditions not satisfied')
        for clause in self.effect:
            kb.tell(self.substitute(clause, args))
            if clause.op[:3] == 'Not':
                new_clause = Expr(clause.op[3:], *clause.args)

                if kb.ask(self.substitute(new_clause, args)) is not False:
                    kb.retract(self.substitute(new_clause, args))
            else:
                new_clause = Expr('Not' + clause.op, *clause.args)

                if kb.ask(self.substitute(new_clause, args)) is not False:
                    kb.retract(self.substitute(new_clause, args))

        return kb


## Ejemplo: Mono y Bananas

Hay un mono en el laboratorio, y quiere bananas. Hay tres ubicaciones
en el laboratorio: A, B y C. El mono está en la ubicación A. Hay una caja
en la ubicación C. Hay bananas en la ubicación B, pero cuelgan del
techo. El mono necesita la caja para alcanzar todas las bananas.

In [ ]:
def monkey_banana():
    return PlanningProblem(initial='MonoEn(A) & NivelMono(abajo) & CajaEn(C) & BananasEn(B)', 
                           goals='MonoTiene(bananas)',
                           actions=[Action('Ir(x, y)',
                                           precond='MonoEn(x) & NivelMono(abajo)',
                                           effect='~MonoEn(x) & MonoEn(y)',
                                           domain='Posicion(x) & Posicion(y)'),
                                    Action('Trepar(x)',
                                           precond='MonoEn(x) & NivelMono(abajo) & CajaEn(x)',
                                           effect='~NivelMono(abajo) & NivelMono(arriba)',
                                           domain='Posicion(x)'),
                                    Action('Bajarse(x)',
                                           precond='MonoEn(x) & NivelMono(arriba) & CajaEn(x)',
                                           effect='~NivelMono(arriba) & NivelMono(abajo)',
                                           domain='Posicion(x)'),
                                    Action('LLevarCaja(x,y)',
                                           precond='MonoEn(x) & NivelMono(abajo) & CajaEn(x)',
                                           effect='~CajaEn(x) & CajaEn(y) & ~MonoEn(x) & MonoEn(y)',
                                           domain='Posicion(x) & Posicion(y)'),
                                    Action('TomarBananas(x)',
                                           precond='MonoEn(x) & NivelMono(arriba) & BananasEn(x) & CajaEn(x)',
                                           effect='MonoTiene(bananas)',
                                           domain='Posicion(x)'),],
                           domain='Posicion(A) & Posicion(B) & Posicion(C)')

In [ ]:
def get_solution(problem):
    solution = uniform_cost_search(ForwardPlan(problem)).solution()
    solution = list(map(lambda action: Expr(action.name, *action.args), solution))
    return solution

In [ ]:
get_solution(monkey_banana())

## Problema 1: Carga aérea

En el problema de la carga aérea, comenzamos con la carga en dos aeropuertos, A y B. Nuestro objetivo es enviar cada carga al otro aeropuerto. Tenemos dos aviones para ayudarnos a realizar la tarea.
Tenemos la carga C1, y el avión P1 en el aeropuerto A y la carga C2, y el avión P2 en el aeropuerto B.
Nuestro objetivo es tener la carga C1 en el aeropuerto B y la carga C2 en el aeropuerto A.

In [ ]:
def air_cargo():
    # TODO: Your code here!
    return None

In [ ]:
get_solution(air_cargo())

## Problema 2: La rueda de repuesto

Consideremos el problema de cambiar un neumático desinflado de un automóvil.
El objetivo es montar una rueda de repuesto en el eje del coche, dado que tenemos una rueda pinchada en el eje y una rueda de repuesto en el maletero.

In [ ]:
def spare_tire():
    # TODO: Your code here!
    return None

In [ ]:
get_solution(spare_tire())

## Problema 3: Torre de tres bloques

Este problema consiste en un conjunto de bloques en forma de cubo colocados sobre una mesa.
Los bloques se pueden apilar, pero solo un bloque puede encajar directamente encima de otro.
Un brazo robótico puede levantar un bloque y moverlo a otra posición, ya sea sobre la mesa o encima de otro bloque.
El brazo solo puede levantar un bloque a la vez, por lo que no puede levantar un bloque que tenga otro encima.
El objetivo siempre será construir una o más pilas de bloques.
En nuestro caso, consideramos solo tres bloques.
La configuración particular que usaremos se llama anomalía de Sussman en honor al Prof. Gerry Sussman.

![](blocks.png)

In [ ]:
def three_block_tower():
    # TODO: Your code here!
    return None

In [ ]:
get_solution(three_block_tower())

## Problema 4: Compras

Se tiene una lista de compras para realizar. Los objetos a comprar son leche, plátanos y un taladro.
Inicialmente, partimos de casa y sabemos que en el mercado hay leche y plátanos y en la ferretería venden taladros.

In [ ]:
def shopping_problem():
    # TODO: Your code here!
    return None

In [ ]:
get_solution(shopping_problem())